In [ ]:
# | default_exp _code_generator.app_generator

In [ ]:
# | export

from typing import *
import time
import json
from pathlib import Path

from yaspin import yaspin

from fastkafka_gen._components.logger import get_logger
from fastkafka_gen._code_generator.helper import CustomAIChat, ValidateAndFixResponse, write_file_contents, read_file_contents, validate_python_code
from fastkafka_gen._code_generator.prompts import APP_GENERATION_PROMPT_FROM_ASYNCAPI, APP_SKELETON_GENERATION_PROMPT, APP_GENERATION_PROMPT_FROM_SKELETON
from fastkafka_gen._code_generator.constants import ASYNC_API_SPEC_FILE_NAME, APPLICATION_FILE_NAME, DESCRIPTION_FILE_NAME, \
                                                    APPLICATION_SKELETON_FILE_NAME, GENERATE_APP_FROM_ASYNCAPI, \
                                                    GENERATE_APP_FROM_SKELETON, GENERATE_APP_SKELETON

In [ ]:

from tempfile import TemporaryDirectory

from fastkafka_gen._components.logger import suppress_timestamps

In [ ]:
# | export

logger = get_logger(__name__)

In [ ]:
suppress_timestamps()
logger = get_logger(__name__, level=20)
logger.info("ok")

[INFO] __main__: ok


In [ ]:
# | export

generate_keys = {
    GENERATE_APP_FROM_ASYNCAPI: 
        {
            "prompt": APP_GENERATION_PROMPT_FROM_ASYNCAPI, 
            "input_file": ASYNC_API_SPEC_FILE_NAME, 
            "output_file": APPLICATION_FILE_NAME,
        },
    GENERATE_APP_SKELETON: 
        {
            "prompt": APP_SKELETON_GENERATION_PROMPT,
            "input_file": DESCRIPTION_FILE_NAME,
            "output_file": APPLICATION_SKELETON_FILE_NAME,
        },
    GENERATE_APP_FROM_SKELETON: 
        {
            "prompt": APP_GENERATION_PROMPT_FROM_SKELETON, 
            "input_file": APPLICATION_SKELETON_FILE_NAME, 
            "output_file": APPLICATION_FILE_NAME,
        },
}

In [ ]:
# | export


def generate_app(code_gen_directory: str, total_usage: List[Dict[str, int]], generate_key:str = GENERATE_APP_FROM_ASYNCAPI) -> List[Dict[str, int]]:
    """Generate code for the new FastKafka app from the validated plan

    Args:
        code_gen_directory: The directory containing the generated files.
        total_usage: list of token usage
        generate_key: key which indicates which prompt will be used for the app creation
    Returns:
        The total token used to generate the FastKafka code
    """
    assert generate_key in generate_keys
    additional_text = "skeleton " if generate_key == GENERATE_APP_SKELETON else ""
    
    with yaspin(
        text=f"Generating FastKafka app {additional_text}(usually takes around 15 to 30 seconds)...", color="cyan", spinner="clock"
    ) as sp:
        file_name = f"{code_gen_directory}/{generate_keys[generate_key]['input_file']}"
        file_content = read_file_contents(file_name)

        prompt = generate_keys[generate_key]["prompt"]
        # Without the following line, GPT sometimes returns text instead of python script
        prompt += "\n==== YOUR RESPONSE ====\n"
        app_generator = CustomAIChat(
            params={
                "temperature": 0.5,
            },
            user_prompt=prompt,
        )
        app_validator = ValidateAndFixResponse(app_generator, validate_python_code)
        validated_app, total_usage = app_validator.fix(file_content, total_usage)

        output_file = f"{code_gen_directory}/{generate_keys[generate_key]['output_file']}"
        write_file_contents(output_file, validated_app)

        sp.text = ""
        sp.ok(f" ✔ FastKafka app {additional_text}generated and saved at: {output_file}")
        return total_usage

In [ ]:
# | notest

fixture_spec = '''
asyncapi: 2.5.0
info:
  title: Product currency converter
  version: 0.0.1
  description: 'A FastKafka application using localhost broker for testing, staging.airt.ai
    for staging and prod.airt.ai for production, using default port numbers. It should
    consume from ''store_product'' topic an JSON encoded object with the following
    three attributes: product_name, currency and price. The format of the currency
    will be three letter string, e.g. ''EUR''. For each consumed message, check if
    the currency attribute is set to ''HRK''. If it is then change the currency to
    ''EUR'' and divide the price by 7.5, if the currency is not set to ''HRK'' don''t
    change the original message. Finally, publish the consumed message to ''change_currency''
    topic. Use SASL_SSL with SCRAM-SHA-256 for authentication.'
  contact:
    name: Author
    url: https://www.google.com/
    email: noreply@gmail.com
servers:
  localhost:
    url: localhost
    description: local development kafka broker
    protocol: kafka
    variables:
      port:
        default: '9092'
  staging:
    url: staging.airt.ai
    description: staging kafka broker
    protocol: kafka-secure
    security:
    - staging_default_security: []
    variables:
      port:
        default: '9092'
  production:
    url: prod.airt.ai
    description: production kafka broker
    protocol: kafka-secure
    security:
    - production_default_security: []
    variables:
      port:
        default: '9092'
channels:
  store_product:
    subscribe:
      message:
        $ref: '#/components/messages/StoreProduct'
      description: For each consumed message, check if the currency attribute is set
        to 'HRK'. If it is then change the currency to 'EUR', then make a new instance of the message and only change the price attribute
        by dividing it by 7.5, if the currency is not set to 'HRK' don't change the original message.
        Finally, publish the consumed message to 'change_currency' topic.
  change_currency:
    publish:
      message:
        $ref: '#/components/messages/StoreProduct'
components:
  messages:
    StoreProduct:
      payload:
        properties:
          product_name:
            description: Name of the product.
            title: Product Name
            type: string
          currency:
            description: The currency.
            title: Currency
            type: string
          price:
            description: Price of the product.
            title: Price
            type: number
        required:
        - product_name
        - currency
        - price
        title: StoreProduct
        type: object
  schemas: {}
  securitySchemes:
    staging_default_security:
      type: scramSha256
    production_default_security:
      type: scramSha256

'''

with TemporaryDirectory() as d:
    output_path = f"{str(d)}/fastkafka-gen"
    output_file = f"{output_path}/{APPLICATION_FILE_NAME}"
    spec_file = f"{output_path}/{ASYNC_API_SPEC_FILE_NAME}"    
    write_file_contents(spec_file, fixture_spec)
    
    usage = generate_app(output_path, [])
    
    assert Path(output_path).exists()
    
    actual = [file for file in Path(output_path).iterdir()]
    print(actual)
    assert len(actual) == 2
    
    contents = read_file_contents(output_file)
    print(contents)

assert int(usage[0]["total_tokens"]) > 0
print(usage)

⠹ Generating FastKafka app (usually takes around 15 to 30 seconds)... 

/home/robert/.local/lib/python3.11/site-packages/yaspin/core.py:120: UserWarning: color, on_color and attrs are not supported when running in jupyter
  self._color = self._set_color(color) if color else color


 ✔ FastKafka app generated and saved at: /tmp/tmp0zwat8dy/fastkafka-gen/application.py 
[PosixPath('/tmp/tmp0zwat8dy/fastkafka-gen/application.py'), PosixPath('/tmp/tmp0zwat8dy/fastkafka-gen/asyncapi.yml')]
from typing import *
from pydantic import BaseModel, Field
from aiokafka.helpers import create_ssl_context

from fastkafka import FastKafka


class StoreProduct(BaseModel):
    product_name: str = Field(..., description="Name of the product.")
    currency: str = Field(..., description="The currency.")
    price: float = Field(..., description="Price of the product.")

kafka_brokers = {
    "localhost": {
        "url": "localhost",
        "description": "local development kafka broker",
        "port": 9092,
    },
    "staging": {
        "url": "staging.airt.ai",
        "description": "staging kafka broker",
        "port": 9092,
        "protocol": "kafka-secure",
        "security": {"type": "scramSha256"},
    },
    "production": {
        "url": "prod.airt.ai",
        "de

In [ ]:
# | notest

fixture_description = """
Generate fastkafka app with ono consumes and three produces methods.
The app consumes the messages from the email topic. While consuming it produces messages to welcome, introduction and tutorial topics. Each produces function returns Email 
"""
                
with TemporaryDirectory() as d:
    output_path = f"{str(d)}/fastkafka-gen"
    output_file = f"{output_path}/{APPLICATION_SKELETON_FILE_NAME}"
    description_file = f"{output_path}/{DESCRIPTION_FILE_NAME}"    
    write_file_contents(description_file, fixture_description)
    
    usage = generate_app(output_path, [], GENERATE_APP_SKELETON)
    
    assert Path(output_path).exists()
    
    actual = [file for file in Path(output_path).iterdir()]
    print(actual)
    assert len(actual) == 2
    
    contents = read_file_contents(output_file)
    print(contents)

assert int(usage[0]["total_tokens"]) > 0
print(usage)

 ✔ FastKafka app skeleton generated and saved at: /tmp/tmpvfhj2w7y/fastkafka-gen/application_skeleton.py 
[PosixPath('/tmp/tmpvfhj2w7y/fastkafka-gen/app_description.txt'), PosixPath('/tmp/tmpvfhj2w7y/fastkafka-gen/application_skeleton.py')]
from typing import *
from pydantic import BaseModel, Field
from fastkafka import FastKafka


class Email(BaseModel):
    subject: str = Field(..., description="Subject of the email")
    body: str = Field(..., description="Body of the email")
    recipient: str = Field(..., description="Recipient of the email")


kafka_brokers = {
    "localhost": {
        "url": "localhost",
        "description": "local development kafka broker",
        "port": 9092,
    }
}

app_description = "A FastKafka application that consumes messages from the 'email' topic. It then produces messages to the 'welcome', 'introduction', and 'tutorial' topics. Each produced message contains attributes like 'subject', 'body', and 'recipient'."

app = FastKafka(
    kafka_broker

In [ ]:
# | notest

fixture_app_skeleton = """
from typing import *
from pydantic import BaseModel, Field
from fastkafka import FastKafka


class Email(BaseModel):
    subject: str = Field(..., description="Subject of the email")
    body: str = Field(..., description="Body of the email")
    recipient: str = Field(..., description="Recipient of the email")
    sender: str = Field(..., description="Sender of the email")

kafka_brokers = {
    "localhost": {
        "url": "localhost",
        "description": "local development kafka broker",
        "port": 9092,
    }
}

app_description = "A FastKafka application that consumes messages from the 'email' topic. Each message should include attributes like 'subject', 'body', 'recipient', and 'sender'. After consuming, the application produces messages to the 'welcome', 'introduction', and 'tutorial' topics."

app = FastKafka(
    kafka_brokers=kafka_brokers,
    description=app_description,
    title='Email App',
)

consume_description = "Consume messages from the 'email' topic and produce messages to the 'welcome', 'introduction', and 'tutorial' topics."
@app.consumes(topic="email", description=consume_description)
async def on_email(msg: Email):
    raise NotImplementedError()


welcome_description = "Produce messages to the 'welcome' topic."
@app.produces(topic="welcome", description=welcome_description)
async def to_welcome(msg: Email) -> Email:
    raise NotImplementedError()


introduction_description = "Produce messages to the 'introduction' topic."
@app.produces(topic="introduction", description=introduction_description)
async def to_introduction(msg: Email) -> Email:
    raise NotImplementedError()


tutorial_description = "Produce messages to the 'tutorial' topic."
@app.produces(topic="tutorial", description=tutorial_description)
async def to_tutorial(msg: Email) -> Email:
    raise NotImplementedError()
"""
                
with TemporaryDirectory() as d:
    output_path = f"{str(d)}/fastkafka-gen"
    output_file = f"{output_path}/{APPLICATION_FILE_NAME}"
    app_skeleton_file = f"{output_path}/{APPLICATION_SKELETON_FILE_NAME}"    
    write_file_contents(app_skeleton_file, fixture_app_skeleton)
    
    usage = generate_app(output_path, [], GENERATE_APP_FROM_SKELETON)
    
    assert Path(output_path).exists()
    
    actual = [file for file in Path(output_path).iterdir()]
    print(actual)
    assert len(actual) == 2
    
    contents = read_file_contents(output_file)
    print(contents)

assert int(usage[0]["total_tokens"]) > 0
print(usage)

 ✔ FastKafka app generated and saved at: /tmp/tmp894n_k37/fastkafka-gen/application.py 
[PosixPath('/tmp/tmp894n_k37/fastkafka-gen/application_skeleton.py'), PosixPath('/tmp/tmp894n_k37/fastkafka-gen/application.py')]
from typing import *
from pydantic import BaseModel, Field
from fastkafka import FastKafka


class Email(BaseModel):
    subject: str = Field(..., description="Subject of the email")
    body: str = Field(..., description="Body of the email")
    recipient: str = Field(..., description="Recipient of the email")
    sender: str = Field(..., description="Sender of the email")

kafka_brokers = {
    "localhost": {
        "url": "localhost",
        "description": "local development kafka broker",
        "port": 9092,
    }
}

app_description = "A FastKafka application that consumes messages from the 'email' topic. Each message should include attributes like 'subject', 'body', 'recipient', and 'sender'. After consuming, the application produces messages to the 'welcome', 'i